In [ ]:
# -*- coding:utf-8 -*-
import cv2
import facenet
import dlib
import tensorflow as tf
import numpy as np
import os
import pickle
from sklearn.svm import SVC

In [ ]:
face_detector = dlib.get_frontal_face_detector()

In [ ]:
model = 'C:/Users/VIPLAB/models/facenet/20171208_only_3image_8people.pb'
classifier_filename = 'C:/Users/VIPLAB/models/facenet/20171208_only_3image_8people.pkl'
train_data_dir = "../xiao_face_dataset_only_face/"

In [ ]:
classes = os.listdir(train_data_dir)
classes.sort()
classes

# load model

In [ ]:
sess = tf.Session()
facenet.load_model(model)
            
# Get input and output tensors
images_placeholder = tf.get_default_graph().get_tensor_by_name("input:0")
embeddings = tf.get_default_graph().get_tensor_by_name("embeddings:0")
phase_train_placeholder = tf.get_default_graph().get_tensor_by_name("phase_train:0")
embedding_size = embeddings.get_shape()[1]
# Run forward pass to calculate embeddings
print('Calculating features for images')

In [ ]:
classifier_filename_exp = os.path.expanduser(classifier_filename)
with open(classifier_filename_exp, 'rb') as infile:
                    (model, class_names) = pickle.load(infile)
print('Loaded classifier model from file "%s"' % classifier_filename_exp)

In [ ]:
def predict(images, threshold = 0.15):
    results = []
    threshold = threshold
    size = 160
    image_count = len(images)
    resize_images = [cv2.resize(image,(size, size), interpolation = cv2.INTER_CUBIC) for image in images]
    feed_dict = { images_placeholder:resize_images, phase_train_placeholder:False }
    emb_array = np.zeros((image_count, embedding_size))
    emb_array[0:image_count,:] = sess.run(embeddings, feed_dict=feed_dict)
    predictions = model.predict_proba(emb_array)
    best_class_indices = np.argmax(predictions, axis=1)
    best_class_probabilities = predictions[np.arange(len(best_class_indices)), best_class_indices]
    
    for i in range(len(best_class_indices)):
#         print('%4d  %s: %.3f' % (i, class_names[best_class_indices[i]], best_class_probabilities[i]))
        if(best_class_probabilities[i] >= threshold):
#             print(class_names[best_class_indices[i]])
            results.append(class_names[best_class_indices[i]])
        else:
            results.append("unknown")
    return results

# load camera

In [ ]:
cap = cv2.VideoCapture(0)
correct_people = '林詠翔'
correct_color = (0,255,0) # 綠
failed_color = (255,0,0) # 紅


In [ ]:
while True:
    _, frame = cap.read()
    
    if frame is None:
        print("lost camera")
        break

    detected_faces = face_detector(frame, 1)
    facerect = [(x.left(), x.top(),
                    x.right(), x.bottom()) for x in detected_faces]
    facerect_revise = []
    for index, rect in enumerate(facerect):
        x_left = rect[0]
        x_right = rect[2]
        y_top = rect[1]
        y_bottom = rect[3]
#         避免超出邊界
        if(x_left < 0):
            x_left = 0
        if(x_right > width):
            x_right = width-1
        if(y_top < 0):
            y_top = 0
        if(y_bottom > height):
            y_bottom = height-1
        facerect_revise.append(tuple([x_left, y_top, x_right, y_bottom]))
    facerect = facerect_revise
    if len(facerect) > 0:
#         print('face detected')
        
#         color = (255, 255, 255)  # 白
        images = [frame[rect[1]: rect[3], rect[0]: rect[2]] for rect in facerect]
        result = predict(images)
        for index, rect in enumerate(facerect):
            # 找出人臉位置的地方, 並用長方形框起來
                
            if(result[index] != "unknown"):
#                 cv2.rectangle(frame, tuple(rect[0:2]), tuple(rect[2:4]), color, 2)
#                 print(result[index])
                person_name = result[index]
                if(result[index] == correct_people):
                    color = correct_color
                    person_name = "xiaosean"
                else:
                    color = failed_color
                x, y = rect[0], rect[1]
                cv2.rectangle(frame, tuple(rect[0:2]), tuple(rect[2:4]), color, 2)
                cv2.putText(frame, person_name, (x,y + 30),
                        cv2.FONT_HERSHEY_SIMPLEX, fontScale  = 1,
                        color = color,
                        thickness=2)
                
            
    cv2.imshow("face dection", frame)
                
    #延遲 10msec
    k = cv2.waitKey(10)
    #點擊esc離開
    if k == 27:
        break

    #キャプチャを終了
cap.release()
cv2.destroyAllWindows()